In [28]:
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.chat_models import ChatOllama
from langchain_community.document_loaders import JSONLoader

embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
llm = ChatOllama(model="qwen3:0.6b")



In [41]:
loader = JSONLoader(
    file_path="/mnt/c/work/portofolio/warehouse_inventory.json",
    jq_schema=".warehouse.inventory[]",
    text_content=False,
    metadata_func=lambda record, metadata: {
        **metadata,
        **record  # Include all fields from the record
    }
)

In [44]:
docs

[Document(metadata={'source': '/mnt/c/work/portofolio/warehouse_inventory.json', 'seq_num': 1, 'item_id': 'ITM001', 'name': 'Laptop 15-inch', 'category': 'Electronics', 'quantity': 150, 'unit_price': 799.99, 'location': 'Aisle 3, Shelf B2', 'last_updated': '2025-07-10', 'status': 'In Stock', 'supplier': 'TechCorp', 'minimum_stock_level': 50}, page_content='{"item_id": "ITM001", "name": "Laptop 15-inch", "category": "Electronics", "quantity": 150, "unit_price": 799.99, "location": "Aisle 3, Shelf B2", "last_updated": "2025-07-10", "status": "In Stock", "supplier": "TechCorp", "minimum_stock_level": 50}'),
 Document(metadata={'source': '/mnt/c/work/portofolio/warehouse_inventory.json', 'seq_num': 2, 'item_id': 'ITM002', 'name': 'Wireless Mouse', 'category': 'Electronics', 'quantity': 300, 'unit_price': 19.99, 'location': 'Aisle 3, Shelf C1', 'last_updated': '2025-07-12', 'status': 'In Stock', 'supplier': 'TechCorp', 'minimum_stock_level': 100}, page_content='{"item_id": "ITM002", "name":

In [47]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="warehouse_collection",  # Name of the collection
    embedding_function=embeddings,
    persist_directory="/mnt/c/work/portofolio/Chatbot/chromadb",  # Where to save data locally, remove if not necessary
)
vector_store.add_documents(docs)
# vector_store.reset_collection()

['c1627931-da6e-471a-aa72-673068aba1f2',
 '61b370a3-6536-421a-9916-4d80d84e5a7a',
 '3e88ebab-3d8a-455b-b47d-cf6707888ab8',
 '64564453-60a0-4bb2-97b3-151b753594bf',
 'ad922858-9b6d-4ea1-a583-6d607a00efec',
 '623933ab-da8c-48b6-9fc9-3cf26c6536b4']

In [48]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})
results = retriever.invoke("paper??")
results

[Document(id='64564453-60a0-4bb2-97b3-151b753594bf', metadata={'minimum_stock_level': 200, 'supplier': 'PaperWorks', 'last_updated': '2025-07-11', 'source': '/mnt/c/work/portofolio/warehouse_inventory.json', 'name': 'A4 Paper Ream', 'category': 'Stationery', 'item_id': 'ITM004', 'status': 'In Stock', 'location': 'Aisle 5, Shelf D3', 'unit_price': 5.99, 'seq_num': 4, 'quantity': 500}, page_content='{"item_id": "ITM004", "name": "A4 Paper Ream", "category": "Stationery", "quantity": 500, "unit_price": 5.99, "location": "Aisle 5, Shelf D3", "last_updated": "2025-07-11", "status": "In Stock", "supplier": "PaperWorks", "minimum_stock_level": 200}'),
 Document(id='623933ab-da8c-48b6-9fc9-3cf26c6536b4', metadata={'status': 'In Stock', 'quantity': 1000, 'category': 'Packaging', 'seq_num': 6, 'source': '/mnt/c/work/portofolio/warehouse_inventory.json', 'supplier': 'PackSavvy', 'minimum_stock_level': 300, 'name': 'Packing Tape', 'last_updated': '2025-07-12', 'item_id': 'ITM006', 'unit_price': 2.